## Train an STL ANN from scratch
# User settings

In [ ]:
######## User Settings ########

#### 1. Select one or more input variables from:
# 'SAC','Exp','SJR','DICU','Vern','SF_Tide','DXC'

input_var = ['SAC','Exp','SJR','DICU','Vern','SF_Tide','DXC']

# 2. Select ONE output station among:
# 'Emmaton', 'Jersey Point', 'Collinsville', 'Rock Slough', 'Antioch', 'Mallard',
# 'LosVaqueros', 'Martinez', 'MiddleRiver', 'Vict Intake', 'CVP Intake', 'CCFB_OldR'

output_stations=['CVP Intake']

# 3. Specify directory to excel dataset and the helper script (folder name only) ####
google_drive_dir = 'python_ANN'

# 4. Specify whether:
#     -- to train the ANN(s) ==> set to 'no'
#     -- to quickly test the code, in which case ANN won't be well-trained ==> set to 'yes'
is_quick_test = 'yes'

# 5. Specify whether running the code on Colab or a local computer
# NOTE: if set to False, please set option #3: google_drive_dir to the local path of python_ANN
#       e.g., google_drive_dir='/Users/siyuqi/Downloads/Calsim-ANN-master/python_ANN'
running_on_colab = True

###### User Settings Finished ######


# Mount Google Drive to Colab and import useful libraries



In [ ]:
import sys
import os

# Mount Google drive
if running_on_colab:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    xl_path = os.path.join('/content/drive','My Drive',google_drive_dir,"ANN_data.xlsx")
    %tensorflow_version 1.x
    sys.path.append(os.path.join('/content/drive','My Drive',google_drive_dir))
else:
    xl_path = os.path.join(os.path.dirname(google_drive_dir),"ANN_data.xlsx")
    sys.path.append(os.path.join(google_drive_dir))

# import helper functions
from ann_helper import read_data,normalize_in,writeF90,initnw,show_eval


# determine whether to run a quick test or a full training
if 'n' in is_quick_test.lower():
    test_mode = False
else:
    test_mode = True
    print('Running a quick test...')


# Prepare for training


1.   Define hyper-parameters
2.   Read data from excel
3.   Split train/test set



In [ ]:
import tensorflow as tf
import math
import numpy as np
from sklearn.model_selection import train_test_split
import time
from scipy import stats

# sort input and output variables
locs = {'Emmaton':0,'Jersey Point':1,'Collinsville':2,'Rock Slough':3,'Antioch':4,
        'Mallard':5, 'LosVaqueros':6, 'Martinez':7, 'MiddleRiver':8, 'Vict Intake':9,
        'CVP Intake':10, 'CCFB_OldR':11}
abbrev_map = {'rock slough':'ORRSL','rockslough':'ORRSL',
            'emmaton':'EMM','jersey point':'JP','jerseypoint':'JP',
            'antioch':'antioch','collinsville':'CO',
            'mallard':'Mallard','mallard island':'Mallard',
            'los vaqueros':'LosVaqueros','losvaqueros':'LosVaqueros',
            'martinez':'MTZ',
            'middle river':'MidR_intake','middleriver':'MidR_intake',
            'victoria cannal':'Victoria_intake','vict intake':'Victoria_intake',
            'cvp intake':'CVP_intake','clfct forebay':'CCFB',
            'clfct forebay intake':'ccfb_intake','x2':'X2'};

output_stations = sorted(output_stations,key=lambda x: locs[x])


# detect available device: CPU or GPU
device_name = tf.test.gpu_device_name()
if 'gpu' not in device_name.lower():
    print('Found CPU only')
else:
    print('Found GPU at: {}'.format(device_name))


# determine hyper-parameters
input_shape = (1,18*len(input_var))
output_shape = 1
nn_shape = [18*len(input_var),8,2,1] # number of neurons in each layer

# LM optimizer settings
max_fail = 3
init_mu = .05
mu_max = 1e10
target_mse = 0.

# Adam optimizer settings
adam_epochs = 100
batch_size = 32


if test_mode or (device_name != '/device:GPU:0' and running_on_colab):
    epochs = 5
else:
    epochs = 100

train_loc = locs[output_stations[0]]
ann_name = abbrev_map[output_stations[0].lower()]
start = time.time()


# read data from excel
x_data,y_data = read_data(xl_path,input_var,output_stations)
end = time.time()
print("loading data in %.2f seconds" % (end-start) )


# normalize data to 0.1 ~ 0.9
[x_norm,x_slope,x_bias] = normalize_in(x_data)
[y_norm,y_slope,y_bias] = normalize_in(y_data)


# split 80% for training, 20% for testing
x_train_ori, x_test_ori, y_train0, y_test0 = train_test_split(x_norm,
                                                              y_norm,
                                                              test_size=0.2,
                                                              random_state = 0)

if test_mode:
    x_train_ori = x_train_ori[:100]
    x_test_ori = x_test_ori[:100]
    y_train0 = y_train0[:100]
    y_test0 = y_test0[:100]


train_err = []
test_err = []
train_shape = len(x_train_ori)

# Prepare the ANN


1.   Build and connect ANN layers
2.   Define cost function
3.   Create an Adam optimizer object
4.   Create an LM optimizer object and implement the LM algorithm 



In [ ]:
# Define inputs, outputs and weights for the ANN, and build ANN using these variables
tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(1)

x = tf.compat.v1.placeholder(tf.float32, [None, 18*len(input_var)], name='InputData')
y = tf.compat.v1.placeholder(tf.float32, [None, 1], name='LabelData')

init_val = initnw(list(zip(*(nn_shape[i:] for i in range(2)))),x_train_ori)

W1 = tf.Variable(initial_value=init_val[0][0], name='w1',dtype='float32')
b1 = tf.Variable(initial_value=init_val[0][1], name='b1',dtype='float32')
W2 = tf.Variable(initial_value=init_val[1][0], name='w2',dtype='float32')
b2 = tf.Variable(initial_value=init_val[1][1], name='b2',dtype='float32')
W3 = tf.Variable(initial_value=init_val[2][0], name='w3',dtype='float32')
b3 = tf.Variable(initial_value=init_val[2][1], name='b3',dtype='float32')

with tf.compat.v1.name_scope('layer1'):
    first_out = tf.sigmoid(tf.add(tf.matmul(x,W1),b1))
with tf.compat.v1.name_scope('layer2'):
    second_out = tf.sigmoid(tf.add(tf.matmul(first_out,W2),b2))
with tf.compat.v1.name_scope('layer3'):
    pred = tf.matmul(second_out, W3) + b3

# r is residuals between target output and ANN estimations
r = tf.subtract(pred,y)

# Cost/loss function, which must be mse or sse for LM optimizer
cost = tf.reduce_mean(input_tensor=tf.square(tf.subtract(pred,y)))


# Define useful variables for adam optimizer:
global_step = tf.Variable(0, trainable=False)
initial_lr = 1e-2
learning_rate = tf.compat.v1.train.exponential_decay(initial_lr, global_step,
                                       40, 0.999, staircase=True)
# Build Adam optimizer
with tf.compat.v1.name_scope('train_op'):
    adam_opt = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost,global_step = global_step)

# Build LM optimizer
with tf.compat.v1.name_scope('train_op'):
    opt = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=1)

with tf.compat.v1.name_scope('Accuracy'):
    acc = tf.reduce_mean(input_tensor=tf.square(tf.subtract(pred,y)))

# Initialize the variables
init = tf.compat.v1.global_variables_initializer()


# Implement LM algorithm
def jacobian(y, x):
    loop_vars = [
        tf.constant(0, tf.int32),
        tf.TensorArray(tf.float32, size=train_shape),
    ]

    _, jacobian = tf.while_loop(
        cond=lambda i, _: i < train_shape,
        body=lambda i, res: (i+1, res.write(i, tf.reshape(tf.gradients(ys=y[i], xs=x), (-1,)))),
        loop_vars=loop_vars)
    return jacobian.stack()


r_flat = tf.expand_dims(tf.reshape(r,[-1]),1)
parms = [W1, b1, W2, b2, W3, b3]
parms_sizes = [tf.size(input=p) for p in parms]
j = tf.concat([jacobian(r_flat, p) for p in parms], 1)
jT = tf.transpose(a=j)
hess_approx = tf.matmul(jT, j)
grad_approx = tf.matmul(jT, r_flat)

mu = tf.compat.v1.placeholder(tf.float32, shape=[])

store = [tf.Variable(tf.zeros(p.shape, dtype=tf.float32)) for p in parms]
save_parms = [tf.compat.v1.assign(s, p) for s, p in zip(store, parms)]
restore_parms = [tf.compat.v1.assign(p, s) for s, p in zip(store, parms)]

wb_flat = tf.concat([tf.reshape(p,[-1,1]) for p in parms],axis=0)
n = tf.add_n(parms_sizes)
I = tf.eye(n, dtype=tf.float32)
w_2 = tf.reduce_sum(input_tensor=tf.square(wb_flat))


sess = tf.compat.v1.Session()
sess.run(init)

dp_flat = tf.matmul(tf.linalg.inv(hess_approx + tf.multiply(mu, I)), grad_approx)
dps = tf.split(dp_flat, parms_sizes, 0)

for i in range(len(dps)):
    dps[i] = tf.reshape(dps[i], parms[i].shape)
lm = opt.apply_gradients(zip(dps, parms))


# print number of parameters
print("Number of trainable parameters: %d" % sess.run(n))

Create a saver object to save trained models

In [ ]:
saver = tf.compat.v1.train.Saver({'W1':W1,'b1':b1,'W2':W2,'b2':b2,'W3':W3,'b3':b3})

# Train the ANN with Adam optimizer 


In [ ]:
# Function that feeds data batches to the ANN
def get_batch(inputX, inputY, batch_size):
    duration = len(inputX)
    for i in range(0,duration//batch_size):
        idx = i*batch_size
        yield inputX[idx:idx+batch_size], inputY[idx:idx+batch_size]



# Start training

start = time.time()
with sess.as_default():
    with sess.graph.as_default():
        cost_list = []
        lr_list = []
        # Run the initializer
        sess.run(init)

        # Training cycle
        total_batch = train_shape//batch_size
        for epoch in range(adam_epochs):
            avg_cost = 0.
            ii = 0
            # Loop over all batches
            for batch_xs, batch_ys in get_batch(x_train_ori, y_train0,batch_size):
                ii = ii +1
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c = sess.run([adam_opt, cost],
                                 feed_dict={x: batch_xs, y: batch_ys})
                # Write logs at every iteration
                # Compute average loss
                avg_cost += c / total_batch
            lr = learning_rate.eval()
            cost_list.append(avg_cost)
            lr_list.append(lr)
            # Display logs per epoch step
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost),
                  "lr = ","{:.10f}".format(lr))

    print("Adam optimization Finished in %d seconds" %(time.time() - start))

    # Evaluate model performance
    print("Train MSE:", acc.eval({x: x_train_ori, y: y_train0}))
    print("Test MSE:", acc.eval({x: x_test_ori, y: y_test0}))
    y_test_ = sess.run(pred,feed_dict={x:x_test_ori})
    print("Test MAPE:", np.mean(np.abs(y_test_-y_test0)/y_test0))


# Fine-tune the ANN with LM optimizer

In [ ]:
# Define input and output datasets for training
feed_dict = {x: x_train_ori,
             y: y_train0}
feed_dict[mu] = init_mu

# Define input and output datasets for validation
validation_feed_dict = {x: x_test_ori,
                        y: y_test0}

train_break = False
current_loss = sess.run(cost,{x: x_train_ori,y: y_train0})

# Prepare saving directory
if running_on_colab and (not os.path.exists(os.path.join('/content/drive', 'My Drive', google_drive_dir,"models",ann_name))):
    os.makedirs(os.path.join('/content/drive',
                             'My Drive',
                             google_drive_dir,
                             "models",
                             ann_name))


# Start training
start = time.time()
with sess.as_default():
    with sess.graph.as_default():
        epoch = 1
        fail_step = 0
        # Training cycle
        while epoch < epochs and current_loss > target_mse:
            epoch_start = time.time()
            if not(epoch%2) or epoch == 1:
                val_loss = sess.run(cost, validation_feed_dict)
                print('epoch: %3d , mu: %.5e, train loss: %.10f, val loss: %.10f'%(epoch,feed_dict[mu],current_loss,val_loss))
            sess.run(save_parms)
            while True:
                sess.run(lm, feed_dict)
                new_loss = sess.run(cost,feed_dict)
                if new_loss > current_loss:
                    fail_step += 1
                    feed_dict[mu] *= 10
                    if feed_dict[mu] > mu_max or fail_step > max_fail:
                        train_break = True
                        break
                    sess.run(restore_parms)
                else:
                    print("mu: %.5e, new loss: %.5f, previous loss: %.5f"%(feed_dict[mu],new_loss,current_loss))
                    fail_step = 0
                    feed_dict[mu] /= 10
                    feed_dict[mu] = max(1e-20,feed_dict[mu])
                    current_loss = new_loss
                    break
            if train_break:
                print('Failed for %d step(s), current mu = %.5e, stop training' % (fail_step,feed_dict[mu]))
                break
            epoch += 1
            print('Time elapsed: %.1f seconds\n'%(time.time() - epoch_start))
    print("Optimization finished in %d seconds!\n" %(time.time() - start))

    # Evaluate model performance
    y_train_predicted = sess.run(pred,feed_dict)
    y_test_predicted = sess.run(pred,validation_feed_dict)


# Save model to current workspace
model_path = "./%s/model.ckpt" % (ann_name)
save_path = saver.save(sess, model_path)
print("Model saved to current workspace: %s" % os.path.abspath(save_path))

# Save model to Google Drive
if running_on_colab:
    model_path = os.path.join('/content/drive','My Drive',google_drive_dir,"models/%s/model.ckpt"%ann_name)
    save_path = saver.save(sess, model_path)
    print("Model saved to Google Drive: %s" % os.path.abspath(save_path))

# Visualize results
show_eval(y_train_predicted[:,0],
          y_train0[:,0],
          y_test_predicted[:,0],
          y_test0[:,0],
          y_slope,y_bias,ann_name)

# Write weights to fortran file
writeF90(ann_name,abbrev_map[output_stations[0].lower()],
          y_slope[0],y_bias[0],
          sess.run(W1).transpose(),sess.run(b1),
          sess.run(W2).transpose(),sess.run(b2),
          sess.run(W3).transpose(),sess.run(b3))

# Write weights to fortran file to Google Drive
if running_on_colab:
    writeF90(os.path.join('/content/drive','My Drive',google_drive_dir,"models/%s"%ann_name),abbrev_map[output_stations[0].lower()],
            y_slope[0],y_bias[0],
            sess.run(W1).transpose(),sess.run(b1),
            sess.run(W2).transpose(),sess.run(b2),
            sess.run(W3).transpose(),sess.run(b3))

with sess.as_default():
    with sess.graph.as_default():
        saved_test_loss = acc.eval(validation_feed_dict)
        print("Train Error after LM:", acc.eval({x: x_train_ori, y: y_train0}))
        print("Test Error after LM:", saved_test_loss)


# Save true output data and ANN estimations into csv to current working directory

In [ ]:
import pandas as pd
input_data =  x_norm
output_data = y_norm
feed_dict = {x: input_data,y: output_data}

loc = output_stations[0]
results_dir = "./%s" % (ann_name)
google_drive_results_path = os.path.join('/content/drive','My Drive',google_drive_dir,"models/%s"%ann_name)
date = np.arange('1940-10',
                 np.datetime64('1940-10') + np.timedelta64(len(input_data), 'D'),
                 dtype='datetime64[D]')

with sess.as_default():
    with sess.graph.as_default():
        # let ANN compute estimations
        y_predicted = sess.run(pred,feed_dict)
        MSE = np.mean(((y_predicted-output_data)/y_slope[0])**2,axis=0)
        MAPE = np.mean(np.abs(y_predicted-output_data)/output_data,axis=0)
        # print estimation errors
        print("MSE = %d" % MSE)
        print("MAPE = %.2f%%" % (MAPE*100))

# write ANN estimations to csv
results = pd.DataFrame(data=((y_predicted-y_bias[0])/y_slope[0]).reshape(-1,),
                        index=date,
                        columns=[loc])
results.index.name='date    '
results.to_csv(os.path.join(results_dir,'%s_ANN_estimations.txt'%ann_name),
                sep='\t',
                float_format='%5.4f',
                header=True,
                index=True)
# print paths of the csv files
print('-'*65)
print('-'*65)
print("ANN estimations written to: \n%s" % (os.path.abspath(os.path.join(results_dir,'%s_ANN_estimations.txt'%ann_name))))
if running_on_colab:
    results.to_csv(os.path.join(google_drive_results_path,'%s_ANN_estimations.txt'%ann_name),
                    sep='\t',
                    float_format='%5.4f',
                    header=True,
                    index=True)
    print(os.path.join(google_drive_results_path,'%s_ANN_estimations.txt'%ann_name))

print('-'*65)

# write target output values to csv
real_data = pd.DataFrame(data=(output_data/y_slope[0]).reshape(-1,),
                        index=date,
                        columns=[loc])
real_data.index.name='date    '
real_data.to_csv(os.path.join(results_dir,'%s_target_outputs.txt'%ann_name),
                  sep='\t',
                  float_format='%5.4f',
                  header=True,
                  index=True)

# print paths of the csv files
print("True salinity values written to: \n%s" %(os.path.abspath(os.path.join(results_dir,'%s_target_outputs.txt'%ann_name))))

# Also save csv files to Google Drive
if running_on_colab:
    real_data.to_csv(os.path.join(google_drive_results_path,'%s_target_outputs.txt'%ann_name),
                      sep='\t',
                      float_format='%5.4f',
                      header=True,
                      index=True)
    print(os.path.join(google_drive_results_path,'%s_target_outputs.txt'%ann_name))

print('-'*65)
print('-'*65)


# Compress trained model and fortran file (on Colab)

In [ ]:
if running_on_colab:
    from google.colab import files
    ann_zip = ann_name +'.zip'
    !zip -r /content/$ann_zip $ann_name

# Download zip file (from Colab)

In [ ]:
if running_on_colab:
    files.download('/content/%s' % ann_zip)
    # files.download('%s.jpg'%(output_stations[0]+str(fig_index)))